In [11]:
# Required libraries
import sys
import os
import IPython as IP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import pickle
# from include import helpers
from IPython.display import clear_output, display
import soundfile as sf

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [12]:
from google.colab import drive
drive.mount('/content/drive')

!touch '/content/drive/MyDrive/Final/Newerdataset.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import pandas as pd
# to get the id part of the file
import numpy as np
import os
%matplotlib inline

dataset_dir =  '/content/drive/MyDrive/Final/'

metadata = pd.read_csv(dataset_dir+str('Newerdataset.csv'))
def assignNew(label):
  if label == 1 :
    return "snore"
  elif label == 0:
    return "noise"
  elif label == 2:
    return "cough"
  else: 
    return -1

metadata["label"] = metadata['class'].apply(assignNew)
metadata.head()
# metadata.shape

,tittle,class,label
0,0 (1).wav,0,noise
1,0 (10).wav,0,noise
2,0 (100).wav,0,noise
3,0 (101).wav,0,noise
4,0 (102).wav,0,noise


In [14]:
def add_padding(features, mfcc_max_padding=174):
    padded = []

    # Add padding
    for i in range(len(features)):
        px = features[i]
        size = len(px[0])
        # Add padding if required
        if (size < mfcc_max_padding):
            xDiff = mfcc_max_padding - size
            xLeft = xDiff//2
            xRight = xDiff-xLeft
            px = np.pad(px, pad_width=((0,0), (xLeft, xRight)), mode='constant')
        
        padded.append(px)

    return padded

In [15]:
# Iterate through all audio files and extract MFCC
features = []
labels = []
frames_max = 0
counter = 0
total_samples = len(metadata)
mfcc_max_padding=0
# Windowing
n_mfcc = 40
for index, row in metadata.iterrows():
  filename = str(metadata.at[index,'tittle']) ##string of file name
  audioPath = str('/content/drive/MyDrive/Final/Test /' + filename)
  class_label = row["label"]
########################################
#Mel-Frequency Cepstral Coefficients (MFCC) 
#normalize -> Spectrum stft -->MFCC -> Mel bank -> 
  x, sr = librosa.load(audioPath, sr= None)
  # Normalize audio data between -1 and 1
  try:
    normalized_x = librosa.util.normalize(x)
  except ValueError:  #raised if `y` is empty.
    pass

  # Compute MFCC coefficients
  mfcc = librosa.feature.mfcc(normalized_x, sr=sr, n_mfcc=n_mfcc)

  # Normalize MFCC between -1 and 1
  normalized_mfcc = librosa.util.normalize(mfcc) 
  # Should we require padding
  shape = normalized_mfcc.shape[1]
  if (mfcc_max_padding > 0 & shape < mfcc_max_padding):
            xDiff = mfcc_max_padding - shape
            xLeft = xDiff//2
            xRight = xDiff-xLeft
            normalized_mfcc = np.pad(normalized_mfcc, pad_width=((0,0), (xLeft, xRight)), mode='constant')
  
  num_frames = normalized_mfcc.shape[1]
  # Add row (feature / label)
  features.append(mfcc)
  labels.append(class_label)
  
 # Update frames maximum
  if (num_frames > frames_max):
    frames_max = num_frames
  padded_features = add_padding(features, frames_max)  
  Xnp = np.array(padded_features)
  ynp = np.array(labels)
  # Optionally save the features to disk
  np.save("/content/drive/MyDrive/Final/NX3-mfcc", Xnp)
  np.save("/content/drive/MyDrive/Final/Ny3-mfcc", ynp)

print("you can rest now")
  ################################################################

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")

you can rest now
